In [0]:
import pyspark

In [0]:
# https://towardsdatascience.com/the-most-complete-guide-to-pyspark-dataframes-2702c343b2e8
# https://confizlimited.udemy.com/course/spark-and-python-for-big-data-with-pyspark/learn/lecture/6688224#overview

In [0]:
sdf = sqlContext.sql('select * from people_table')

In [0]:
# in local setup you'll have to make a SparkSession object as follows:
# However in Databricks, a SparkSession object names, spark is already available

"""
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Basics").getOrCreate() #spark session is the driver, each spark application has only one spark session
"""

# to create a spark project from notebook:
"""

if __name__ == '__main__':
  spark = SparkSession.builder.master('local[3]').appName('something').getOrCreate() # local[3] means, use local multi-threaded JVM with 3 threads
  logger = Log4j(spark)
  
  
  ......after doing your work, you close the driver as:
  spark.stop()
"""


Out[4]: "\n\nif __name__ == '__main__':\n spark = SparkSession.builder.master('local[3]').appName('something').getOrCreate() # local[3] means, use local multi-threaded JVM with 3 threads\n logger = Log4j(spark)\n \n \n ......after doing your work, you close the driver as:\n spark.stop()\n"

In [0]:
# read write collect and show are actions, all other things are transformations
# no transformations are performed unless you call an action

"""
There are two types of transformations:
_Narrow dependency (the result is not affected by partition e.g. where age > 40)
_Wide dependency (the result is affected by partition e.g. group by country ) this groupby will result in an internal shuffle-sort repartitioning, you need to do config to make sure that shuffle-sort happens as per your plan
use: spark.sql.shuffle.partitions


repartition your data by using sdf.repartition(number_of_partitions) # remember repartition is a transformation i.e. nothing will happen unless you call an action
"""

Out[5]: '\nThere are two types of transformations:\n_Narrow dependency (the result is not affected by partition e.g. where age > 40)\n_Wide dependency (the result is affected by partition e.g. group by country ) this groupby will result in an internal shuffle-sort repartitioning, you need to do config to make sure that shuffle-sort happens as per your plan\nuse: spark.sql.shuffle.partitions\n\n\nrepartition your data by using sdf.repartition(number_of_partitions) # remember repartition is a transformation i.e. nothing will happen unless you call an action\n'

In [0]:
"""
each spark action causes a job (infering file schema is an action as well)
each job has multiple stages, stages are divided by repartition
each stage has multiple tasks

e.g. 

repartitioned_df = df.repartition(number_of_partitions)
new_df = df.select('column1', 'column2', 'column3').filter('column1>40').groupBy('column3').collect() 

All of this code will cause a single job in spark

"""

Out[6]: "\neach spark action causes a job (infering file schema is an action as well)\neach job has multiple stages, stages are divided by repartition\neach stage has multiple tasks\n\ne.g. \n\nrepartitioned_df = df.repartition(number_of_partitions)\nnew_df = df.select('column1', 'column2', 'column3').filter('column1>40').groupBy('column3').collect() \n\nAll of this code will cause a single job in spark\n\n"

In [0]:
"""
There are two way to execute program on spark:
  1) spark interactive client: spark-shell and notebook
  2) submit job: spark-submit, databricks notebook, rest API
  
_____

batch job vs stream job

______

spark follows a master-slave architecture
cluster has its own master and slave. While inside cluster, driver and executors act as master and slaves as well
when you submit a job, the spark engine will request the cluster manager to allocate a container in which the driver will run. then driver asks the cluster manager for more containers where executors will run

_____

There are different types of CLUSTER MANAGERS which can work with spark
>local (this is just a simulated cluster manager which runs locally)
>YARN
>Kubernetes
>Mesos
>Standalone


______

local[3] means three threads on local cluster. one will be used for driver and other two for executors

______

spark runs in EXECUTION MODES:
1) client : driver on client machine and executors on cluster
2) cluster : driver and executors on cluster

________

EXECUTION TOOLS:
1) IDE, Notebook
2) Job submit


___________

local+client mode+notebook/shell

yarn+client mode+notebook/shell

yarn+cluster mode+submit
"""


Out[7]: '\nThere are two way to execute program on spark:\n 1) spark interactive client: spark-shell and notebook\n 2) submit job: spark-submit, databricks notebook, rest API\n \n_____\n\nbatch job vs stream job\n\n______\n\nspark follows a master-slave architecture\ncluster has its own master and slave. While inside cluster, driver and executors act as master and slaves as well\nwhen you submit a job, the spark engine will request the cluster manager to allocate a container in which the driver will run. then driver asks the cluster manager for more containers where executors will run\n\n_____\n\nThere are different types of CLUSTER MANAGERS which can work with spark\n>local (this is just a simulated cluster manager which runs locally)\n>YARN\n>Kubernetes\n>Mesos\n>Standalone\n\n\n______\n\nlocal[3] means three threads on local cluster. one will be used for driver and other two for executors\n\n______\n\nspark runs in EXECUTION MODES:\n1) client : driver on client machine and executors on cluster\n2) cluster : driver and executors on cluster\n\n________\n\nEXECUTION TOOLS:\n1) IDE, Notebook\n2) Job submit\n\n\n___________\n\nlocal+client mode+notebook/shell\n\nyarn+client mode+notebook/shell\n\nyarn+cluster mode+submit\n'

In [0]:
sdf.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [0]:
# how to define schema manually

from pyspark.sql.types import (StructField, StructType, IntegerType, StringType)

data_schema = [StructField('age', IntegerType(), True),
              StructField('name', StringType(), True)]

final_struct = StructType(fields=data_schema)

# now read the json file with this schema

sdf2 = spark.read.json('dbfs:/FileStore/tables/people_1_.json', schema=final_struct)
sdf2.printSchema()

root
-- age: integer (nullable = true)
-- name: string (nullable = true)

In [0]:
%fs

ls dbfs:/FileStore/shared_uploads/mariashaukat352@gmail.com/

path,name,size
dbfs:/FileStore/shared_uploads/mariashaukat352@gmail.com/appl_stock_1_-1.csv,appl_stock_1_-1.csv,143130
dbfs:/FileStore/shared_uploads/mariashaukat352@gmail.com/appl_stock_1_.csv,appl_stock_1_.csv,143130
dbfs:/FileStore/shared_uploads/mariashaukat352@gmail.com/features.csv,features.csv,592289
dbfs:/FileStore/shared_uploads/mariashaukat352@gmail.com/people_1_.json,people_1_.json,73
dbfs:/FileStore/shared_uploads/mariashaukat352@gmail.com/train.csv,train.csv,12842546


In [0]:
sdf['age'] # this will return a column
sdf.select('age') # this will return a dataframe
sdf.select(['age','name'])

Out[10]: DataFrame[age: bigint, name: string]

In [0]:
sdf.head(2) # list of first two rows as row objects

Out[11]: [Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [0]:
# adding new column (temporarily)

sdf.withColumn('double_age', sdf['age']*2).show()

+----+-------+----------+
 age| name|double_age|
+----+-------+----------+
null|Michael| null|
 30| Andy| 60|
 19| Justin| 38|
+----+-------+----------+

In [0]:
sdf.createOrReplaceTempView('people_table') # making a queriable table from dataframe
# now we can query it using sql

In [0]:
%sql

select  * from people_table

age,name
null,Michael
30,Andy
19,Justin


In [0]:
spark.sql('select * from people_table').show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [0]:
# from pyspark.sql import SparkSession # to be done in local setup only
# spark = SparkSession.builder.appName('ops').getOrCreate()  # to be done in local setup only

sdf_apple = spark.read.csv('dbfs:/FileStore/shared_uploads/mariashaukat352@gmail.com/appl_stock_1_.csv', inferSchema=True, header=True)
sdf_apple.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
 Date| Open| High| Low| Close| Volume| Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
2010-01-04| 213.429998| 214.499996|212.38000099999996| 214.009998|123432400| 27.727039|
2010-01-05| 214.599998| 215.589994| 213.249994| 214.379993|150476200|27.774976000000002|
2010-01-06| 214.379993| 215.23| 210.750004| 210.969995|138040000|27.333178000000004|
2010-01-07| 211.75| 212.000006| 209.050005| 210.58|119282800| 27.28265|
2010-01-08| 210.299994| 212.000006|209.06000500000002|211.98000499999998|111902700| 27.464034|
2010-01-11|212.79999700000002| 213.000002| 208.450005|210.11000299999998|115557400| 27.221758|
2010-01-12|209.18999499999998|209.76999500000002| 206.419998| 207.720001|148614900| 26.91211|
2010-01-13| 207.870005|210.92999500000002| 204.099998| 210.650002|151473000| 27.29172|
2010-01-14|210.11000299999998|210.45999700000002| 209.020004| 209.43|108223500| 27.133657|
2010-01-15|210.92999500000002|211.59999700000003| 205.869999| 205.93|148516900|26.680197999999997|
2010-01-19| 208.330002|215.18999900000003| 207.240004| 215.039995|182501900|27.860484999999997|
2010-01-20| 214.910006| 215.549994| 209.500002| 211.73|153038200| 27.431644|
2010-01-21| 212.079994|213.30999599999998| 207.210003| 208.069996|152038600| 26.957455|
2010-01-22|206.78000600000001| 207.499996| 197.16| 197.75|220441900| 25.620401|
2010-01-25|202.51000200000001| 204.699999| 200.190002| 203.070002|266424900|26.309658000000002|
2010-01-26|205.95000100000001| 213.710005| 202.580004| 205.940001|466777500| 26.681494|
2010-01-27| 206.849995| 210.58| 199.530001| 207.880005|430642100|26.932840000000002|
2010-01-28| 204.930004| 205.500004| 198.699995| 199.289995|293375600|25.819922000000002|
2010-01-29| 201.079996| 202.199995| 190.250002| 192.060003|311488100| 24.883208|
2010-02-01|192.36999699999998| 196.0|191.29999899999999| 194.729998|187469100| 25.229131|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
only showing top 20 rows

In [0]:
sdf_apple.filter('close < 500').show() # using filter with sql
sdf_apple.filter(sdf_apple['close'] < 500).show() # doing without sql
sdf_apple.filter( (sdf_apple['close'] < 500) & (sdf_apple['open'] > 200)) .show() # using & and |


+----------+------------------+------------------+------------------+------------------+---------+------------------+
 Date| Open| High| Low| Close| Volume| Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
2010-01-04| 213.429998| 214.499996|212.38000099999996| 214.009998|123432400| 27.727039|
2010-01-05| 214.599998| 215.589994| 213.249994| 214.379993|150476200|27.774976000000002|
2010-01-06| 214.379993| 215.23| 210.750004| 210.969995|138040000|27.333178000000004|
2010-01-07| 211.75| 212.000006| 209.050005| 210.58|119282800| 27.28265|
2010-01-08| 210.299994| 212.000006|209.06000500000002|211.98000499999998|111902700| 27.464034|
2010-01-11|212.79999700000002| 213.000002| 208.450005|210.11000299999998|115557400| 27.221758|
2010-01-12|209.18999499999998|209.76999500000002| 206.419998| 207.720001|148614900| 26.91211|
2010-01-13| 207.870005|210.92999500000002| 204.099998| 210.650002|151473000| 27.29172|
2010-01-14|210.11000299999998|210.45999700000002| 209.020004| 209.43|108223500| 27.133657|
2010-01-15|210.92999500000002|211.59999700000003| 205.869999| 205.93|148516900|26.680197999999997|
2010-01-19| 208.330002|215.18999900000003| 207.240004| 215.039995|182501900|27.860484999999997|
2010-01-20| 214.910006| 215.549994| 209.500002| 211.73|153038200| 27.431644|
2010-01-21| 212.079994|213.30999599999998| 207.210003| 208.069996|152038600| 26.957455|
2010-01-22|206.78000600000001| 207.499996| 197.16| 197.75|220441900| 25.620401|
2010-01-25|202.51000200000001| 204.699999| 200.190002| 203.070002|266424900|26.309658000000002|
2010-01-26|205.95000100000001| 213.710005| 202.580004| 205.940001|466777500| 26.681494|
2010-01-27| 206.849995| 210.58| 199.530001| 207.880005|430642100|26.932840000000002|
2010-01-28| 204.930004| 205.500004| 198.699995| 199.289995|293375600|25.819922000000002|
2010-01-29| 201.079996| 202.199995| 190.250002| 192.060003|311488100| 24.883208|
2010-02-01|192.36999699999998| 196.0|191.29999899999999| 194.729998|187469100| 25.229131|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
only showing top 20 rows

+----------+------------------+------------------+------------------+------------------+---------+------------------+
 Date| Open| High| Low| Close| Volume| Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
2010-01-04| 213.429998| 214.499996|212.38000099999996| 214.009998|123432400| 27.727039|
2010-01-05| 214.599998| 215.589994| 213.249994| 214.379993|150476200|27.774976000000002|
2010-01-06| 214.379993| 215.23| 210.750004| 210.969995|138040000|27.333178000000004|
2010-01-07| 211.75| 212.000006| 209.050005| 210.58|119282800| 27.28265|
2010-01-08| 210.299994| 212.000006|209.06000500000002|211.98000499999998|111902700| 27.464034|
2010-01-11|212.79999700000002| 213.000002| 208.450005|210.11000299999998|115557400| 27.221758|
2010-01-12|209.18999499999998|209.76999500000002| 206.419998| 207.720001|148614900| 26.91211|
2010-01-13| 207.870005|210.92999500000002| 204.099998| 210.650002|151473000| 27.29172|
2010-01-14|210.11000299999998|210.45999700000002| 209.020004| 209.43|108223500| 27.133657|
2010-01-15|210.92999500000002|211.59999700000003| 205.869999| 205.93|148516900|26.680197999999997|
2010-01-19| 208.330002|215.18999900000003| 207.240004| 215.039995|182501900|27.860484999999997|
2010-01-20| 214.910006| 215.549994| 209.500002| 211.73|153038200| 27.431644|
2010-01-21| 212.079994|213.30999599999998| 207.210003| 208.069996|152038600| 26.957455|
2010-01-22|206.78000600000001| 207.499996| 197.16| 197.75|220441900| 25.620401|
2010-01-25|202.51000200000001| 204.699999| 200.190002| 203.070002|266424900|26.309658000000002|
2010-01-26|205.95000100000001| 213.710005| 202.580004| 205.940001|466777500| 26.681494|
2010-01-27| 206.849995| 210.58| 199.530001| 207.880005|430642100|26.932840000000002|
2010-01-28| 20

In [0]:
results = sdf_apple.filter(sdf_apple['close'] > 700).collect()
results

Out[17]: [Row(Date='2012-09-18', Open=699.879997, High=702.329987, Low=696.4199980000001, Close=701.910004, Volume=93375800, Adj Close=91.329593),
 Row(Date='2012-09-19', Open=700.259979, High=703.989998, Low=699.569977, Close=702.100021, Volume=81718700, Adj Close=91.35431700000001),
 Row(Date='2012-09-21', Open=702.409988, High=705.070023, Low=699.3599849999999, Close=700.089989, Volume=142897300, Adj Close=91.09278)]

In [0]:
results[0].asDict()['Close']

Out[18]: 701.910004

In [0]:
# transformations

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

def date_fmt(df, fmt, fld):
  return df.withColumn(fld, to_date(df[fld], fmt))


In [0]:
# instead of loading data from a file here, we create test dataframe


schema = StructType([StructField('name', StringType()), StructField('date', StringType())])

rows = [Row('Maria', '26/09/1995'), Row('Ahsan', '09/09/1985')]
my_rdd = spark.sparkContext.parallelize(rows,2) # paralellization
my_sdf = spark.createDataFrame(my_rdd, schema)
my_sdf.printSchema()

root
-- name: string (nullable = true)
-- date: string (nullable = true)

In [0]:

my_sdf_transformed = date_fmt(my_sdf, 'd/m/y', 'date')
my_sdf_transformed.printSchema()
my_sdf_transformed.show()

root
-- name: string (nullable = true)
-- date: date (nullable = true)

+-----+----------+
 name| date|
+-----+----------+
Maria|1995-01-26|
Ahsan|1985-01-09|
+-----+----------+

In [0]:
%fs

ls databricks-datasets/

path,name,size
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-datasets/COVID/,COVID/,0
dbfs:/databricks-datasets/README.md,README.md,976
dbfs:/databricks-datasets/Rdatasets/,Rdatasets/,0
dbfs:/databricks-datasets/SPARK_README.md,SPARK_README.md,3359
dbfs:/databricks-datasets/adult/,adult/,0
dbfs:/databricks-datasets/airlines/,airlines/,0
dbfs:/databricks-datasets/amazon/,amazon/,0
dbfs:/databricks-datasets/asa/,asa/,0
dbfs:/databricks-datasets/atlas_higgs/,atlas_higgs/,0


In [0]:
%fs
ls dbfs:/databricks-datasets/airlines/part-00000

path,name,size
dbfs:/databricks-datasets/airlines/part-00000,part-00000,67108879


In [0]:
"""
data_schema = Structtype([StructField('Year', IntegerType(), True),
                          StructField('Month', IntegerType(), True),
                          StructField('DayOfMonth', IntegerType(), True),
                          StructField('DayOfWeek', IntegerType(), True),
                          StructField('DepTime', IntegerType(), True),
                          StructField('CRSDepTime', IntegerType(), True),
                          StructField('ArrTime', IntegerType(), True),
                          StructField('CRSArrTime', IntegerType(), True),
                          StructField('UniqueCarrier', StringType(), True),
                          StructField('FlightNum', IntegerType(), True),
                          StructField('TailNum', IntegerType(), True),
                          StructField('ActualElapsedTime', IntegerType(), True),
                          StructField('CRSElapsedTime', IntegerType(), True),
                          StructField('AirTime', IntegerType(), True),
                          StructField('ArrDelay', IntegerType(), True),
                          StructField('DepDelay', IntegerType(), True),
                          StructField('Origin', StringType(), True),
                          StructField('Dest', StringType(), True),
                          StructField('Distance', IntegerType(), True),
                          StructField('TaxiIn', StringType(), True),
                          StructField('TaxiOut', StringType(), True)]),
"""

air_data = spark.read.csv('dbfs:/databricks-datasets/airlines/part-00000', inferSchema=True, header=True)
air_data.printSchema()
air_data.show(10)

root
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- DayofMonth: integer (nullable = true)
-- DayOfWeek: integer (nullable = true)
-- DepTime: string (nullable = true)
-- CRSDepTime: integer (nullable = true)
-- ArrTime: string (nullable = true)
-- CRSArrTime: integer (nullable = true)
-- UniqueCarrier: string (nullable = true)
-- FlightNum: integer (nullable = true)
-- TailNum: string (nullable = true)
-- ActualElapsedTime: string (nullable = true)
-- CRSElapsedTime: integer (nullable = true)
-- AirTime: string (nullable = true)
-- ArrDelay: string (nullable = true)
-- DepDelay: string (nullable = true)
-- Origin: string (nullable = true)
-- Dest: string (nullable = true)
-- Distance: string (nullable = true)
-- TaxiIn: string (nullable = true)
-- TaxiOut: string (nullable = true)
-- Cancelled: integer (nullable = true)
-- CancellationCode: string (nullable = true)
-- Diverted: integer (nullable = true)
-- CarrierDelay: string (nullable = true)
-- WeatherDelay: string (nullable = true)
-- NASDelay: string (nullable = true)
-- SecurityDelay: string (nullable = true)
-- LateAircraftDelay: string (nullable = true)
-- IsArrDelayed: string (nullable = true)
-- IsDepDelayed: string (nullable = true)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
1987| 10| 14| 3| 741| 730| 912| 849| PS| 1451| NA| 91| 79| NA| 23| 11| SAN| SFO| 447| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA| YES| YES|
1987| 10| 15| 4| 729| 730| 903| 849| PS| 1451| NA| 94| 79| NA| 14| -1| SAN| SFO| 447| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA| YES| NO|
1987| 10| 17| 6| 741| 730| 918| 849| PS| 1451| NA| 97| 79| NA| 29| 11| SAN| SFO| 447| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA| YES| YES|
1987| 10| 18| 7| 729| 730| 847| 849| PS| 1451| NA| 78| 79| NA| -2| -1| SAN| SFO| 447| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA| NO| NO|
1987| 10| 19| 1| 749| 730| 922| 849| PS| 1451| NA| 93| 79| NA| 33| 19| SAN| SFO| 447| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA| YES| YES|
1987| 10| 21| 3| 728| 730| 848| 849| PS| 1451| NA| 80| 79| NA| -1| -2| SAN| SFO| 447| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA| NO| NO|
1987| 10| 22| 4| 728| 730| 852| 849| PS| 1451| NA| 84| 79| NA| 3| -2| SAN| SFO| 447| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA| YES| NO|
1987| 10| 23| 5| 731| 730| 902| 849| PS| 1451| NA| 91| 79| NA| 13| 1| SAN| SFO| 447| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA| YES| YES|
1987| 10| 24| 6| 744| 730| 908| 849| PS| 1451| NA| 84| 79| NA| 19| 14| SAN| SFO| 447| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA| YES| YES|
1987| 10| 25| 7| 729| 730| 851| 849| PS| 1451| NA| 82| 79| NA| 2| -1| SAN| SFO| 447| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA| YES| NO|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
only showing top 10 rows

In [0]:
from pyspark.sql.functions import *
# two ways to select a column, column string and column object
air_data.select('Year').show(10)  
air_data.select(air_data.Year)


+----+
Year|
+----+
1987|
1987|
1987|
1987|
1987|
1987|
1987|
1987|
1987|
1987|
+----+
only showing top 10 rows

Out[23]: DataFrame[Year: int]

In [0]:
air_data.select('IsArrDelayed', 'Cancelled') # you can do this with an sql select statement as well
# or you can put an sql expression within select() using exp

air_data.select("Year","Month", "DayOfWeek").show(10)

air_data.select( expr("to_date(concat(Year,'/',Month,'/',DayOfWeek),'y/m/d') as FlightDate")).show(10)

+----+-----+---------+
Year|Month|DayOfWeek|
+----+-----+---------+
1987| 10| 3|
1987| 10| 4|
1987| 10| 6|
1987| 10| 7|
1987| 10| 1|
1987| 10| 3|
1987| 10| 4|
1987| 10| 5|
1987| 10| 6|
1987| 10| 7|
+----+-----+---------+
only showing top 10 rows

+----------+
FlightDate|
+----------+
1987-01-03|
1987-01-04|
1987-01-06|
1987-01-07|
1987-01-01|
1987-01-03|
1987-01-04|
1987-01-05|
1987-01-06|
1987-01-07|
+----------+
only showing top 10 rows

In [0]:
from pyspark.sql import functions as f

air_data.select(f.count("*").alias('count'), 
               f.sum('Distance').alias('sum'),
               f.countDistinct('Dest').alias('distinct_dest')).show(10)

#all this can be done with sql Expression as well

air_data.selectExpr('count(*) as count', 
                   'sum(distance) as sum').show()



+------+------------+-------------+
 count| sum|distinct_dest|
+------+------------+-------------+
645918|3.75768295E8| 234|
+------+------------+-------------+

+------+------------+
 count| sum|
+------+------------+
645918|3.75768295E8|
+------+------------+

In [0]:
help(f.count)

Help on function count in module pyspark.sql.functions:

count(col)
 Aggregate function: returns the number of items in a group.
 
 .. versionadded:: 1.3

In [0]:
air_data.groupBy('Dest').count().show(10)
# lets change MSY into ABC and GEG into XYZ using User Defined Functions

+----+-----+
Dest|count|
+----+-----+
 MSY| 4557|
 GEG| 1541|
 BUR| 3737|
 SNA| 3578|
 GTF| 559|
 GRR| 1389|
 EUG| 630|
 PVD| 1331|
 GSO| 3092|
 OAK| 4307|
+----+-----+
only showing top 10 rows

In [0]:


def change_dest(dest):
  msy = 'MSY'
  geg = 'GEG'
  if dest == msy:
    return 'ABC'
  if dest == geg:
    return 'XYZ'
  else:
    return dest
  
  
change_dest_udf = udf(change_dest, StringType()) #specify function name and its return type to register it to the driver as a UDF

# UDF with column expression
# we need to register the function to the driver
air_data_temp = air_data.withColumn('Dest', change_dest_udf('Dest')) # with column only changes the data in one column of the SDF, you need column name and the change expression/function
air_data_temp.filter(air_data_temp['Dest']=='MSY').show(10)



# UDF with string expression 
# we will need create UDF function as sql function to the catalog as well as driver
spark.udf.register('change_dest_udf', change_dest, StringType()) # change_dest+StringType() is the signature of the function
[print(f) for f in spark.catalog.listFunctions() if 'change_dest_udf' in f.name] # to check if the function has been registered to the catalog
# how to use a udf function as string expression? Here:
air_data_temp2 = air_data.withColumn('Dest', expr("change_dest_udf(Dest)"))
air_data_temp2.filter(air_data_temp2['Dest']=='MSY').show(10) # check if the changed strings exist in the dataframe

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+

Function(name='change_dest_udf', description=None, className='org.apache.spark.sql.UDFRegistration$$Lambda$6797/1753368688', isTemporary=True)
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+

In [0]:
#there are three types-> simple aggregations,grouping aggregation, window aggregation

"""

# boiler plate code
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from lib.logger import Log4j

if __name__=='__main__':
  spark = SparkSession\
  .builder\
  .appName('Aggregation_demo')\
  .master('local[2]').GetOrCreate()

logger = Log4j(spark)

"""







  

Out[29]: "\n\n# boiler plate code\nfrom pyspark.sql import SparkSession\nfrom pyspark.sql import functions as f\nfrom lib.logger import Log4j\n\nif __name__=='__main__':\n spark = SparkSession .builder .appName('Aggregation_demo') .master('local[2]').GetOrCreate()\n\nlogger = Log4j(spark)\n\n"

In [0]:
#creating DataFrame quickly for testing

data_list = [
  ('Maria', 26, 9, 1995), 
  ('Ahsan', 9, 9, 85), 
  ('Bubu', 1, 1, 19)]

raw_sdf = spark.createDataFrame(data_list).toDF('name', 'day', 'month', 'year')

raw_sdf.printSchema()

from pyspark.sql.functions import *
raw_sdf = raw_sdf.withColumn("id", monotonically_increasing_id())
raw_sdf.show()

# monotonically increasing ID, these are unique accorss partitions, but not necessarily 



root
-- name: string (nullable = true)
-- day: long (nullable = true)
-- month: long (nullable = true)
-- year: long (nullable = true)

+-----+---+-----+----+-----------+
 name|day|month|year| id|
+-----+---+-----+----+-----------+
Maria| 26| 9|1995|17179869184|
Ahsan| 9| 9| 85|42949672960|
 Bubu| 1| 1| 19|60129542144|
+-----+---+-----+----+-----------+

In [0]:
# convert year from 2 digit to 4 digit



raw_sdf1 = raw_sdf.withColumn('year', expr("""
case
when year<20 then year+2000
when year<100 then year+1900 
else year
end"""))

# this will also work is year is string instead of int but in string you'll need to do casting:
# incline casting-> cast(year as int)
# schmema casting-> you can cast in withColumn function as 
#raw_sdf = raw_sdf.withColumn('year', expr("""
#case
#when year<20 then year+2000
#when year<100 then year+1900 
#else year
#end""").cast(IntegerType()))


raw_sdf.show()


# case statement in pyspark

raw_sdf2 = raw_sdf.withColumn('year', \
                              when(col('year')<20, col('year')+2000) \
                             .when(col('year')<100, col('year')+1900) \
                             .otherwise(col('year')))

raw_sdf2.show()


+-----+---+-----+----+-----------+
 name|day|month|year| id|
+-----+---+-----+----+-----------+
Maria| 26| 9|1995|17179869184|
Ahsan| 9| 9| 85|42949672960|
 Bubu| 1| 1| 19|60129542144|
+-----+---+-----+----+-----------+

+-----+---+-----+----+-----------+
 name|day|month|year| id|
+-----+---+-----+----+-----------+
Maria| 26| 9|1995|17179869184|
Ahsan| 9| 9|1985|42949672960|
 Bubu| 1| 1|2019|60129542144|
+-----+---+-----+----+-----------+

In [0]:
from pyspark.sql import Row

# make a function that works on rdd rows:
def make_new_row(row):
  row_dict = row.asDict() #convert rdd row to a dictionary
  row_dict['Average'] = (row_dict['High']+row_dict['Low'])/2 # here you can perform a complex operation as well
  # new key:value pair has been added to the row now
  newrow = Row(**row_dict) #convert back to rdd row
  return newrow # return updated rdd row


apple_rdd = sdf_apple.rdd # convert spark dataframe to rdd
new_apple_rdd = apple_rdd.map(lambda row: make_new_row(row)) # map by default works row by row
new_apple_sdf = sqlContext.createDataFrame(new_apple_rdd) # new dataframe is created from spark context
new_apple_sdf.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+------------------+
 Date| Open| High| Low| Close| Volume| Adj Close| Average|
+----------+------------------+------------------+------------------+------------------+---------+------------------+------------------+
2010-01-04| 213.429998| 214.499996|212.38000099999996| 214.009998|123432400| 27.727039| 213.4399985|
2010-01-05| 214.599998| 215.589994| 213.249994| 214.379993|150476200|27.774976000000002|214.41999399999997|
2010-01-06| 214.379993| 215.23| 210.750004| 210.969995|138040000|27.333178000000004| 212.990002|
2010-01-07| 211.75| 212.000006| 209.050005| 210.58|119282800| 27.28265|210.52500550000002|
2010-01-08| 210.299994| 212.000006|209.06000500000002|211.98000499999998|111902700| 27.464034|210.53000550000002|
2010-01-11|212.79999700000002| 213.000002| 208.450005|210.11000299999998|115557400| 27.221758| 210.7250035|
2010-01-12|209.18999499999998|209.76999500000002| 206.419998| 207.720001|148614900| 26.91211| 208.0949965|
2010-01-13| 207.870005|210.92999500000002| 204.099998| 210.650002|151473000| 27.29172| 207.5149965|
2010-01-14|210.11000299999998|210.45999700000002| 209.020004| 209.43|108223500| 27.133657| 209.7400005|
2010-01-15|210.92999500000002|211.59999700000003| 205.869999| 205.93|148516900|26.680197999999997|208.73499800000002|
2010-01-19| 208.330002|215.18999900000003| 207.240004| 215.039995|182501900|27.860484999999997|211.21500150000003|
2010-01-20| 214.910006| 215.549994| 209.500002| 211.73|153038200| 27.431644|212.52499799999998|
2010-01-21| 212.079994|213.30999599999998| 207.210003| 208.069996|152038600| 26.957455| 210.2599995|
2010-01-22|206.78000600000001| 207.499996| 197.16| 197.75|220441900| 25.620401| 202.329998|
2010-01-25|202.51000200000001| 204.699999| 200.190002| 203.070002|266424900|26.309658000000002| 202.4450005|
2010-01-26|205.95000100000001| 213.710005| 202.580004| 205.940001|466777500| 26.681494| 208.1450045|
2010-01-27| 206.849995| 210.58| 199.530001| 207.880005|430642100|26.932840000000002| 205.0550005|
2010-01-28| 204.930004| 205.500004| 198.699995| 199.289995|293375600|25.819922000000002| 202.0999995|
2010-01-29| 201.079996| 202.199995| 190.250002| 192.060003|311488100| 24.883208| 196.2249985|
2010-02-01|192.36999699999998| 196.0|191.29999899999999| 194.729998|187469100| 25.229131|193.64999949999998|
+----------+------------------+------------------+------------------+------------------+---------+------------------+------------------+
only showing top 20 rows

In [0]:
air_data.show(10)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
1987| 10| 12| 1| NA| 1505| NA| 1608| PS| 1453| NA| NA| 63| NA| NA| NA| BUR| OAK| 325| NA| NA| 1| NA| 0| NA| NA| NA| NA| NA| YES| YES|
1987| 10| 12| 1| NA| 1635| NA| 1732| PS| 1454| NA| NA| 57| NA| NA| NA| OAK| BUR| 325| NA| NA| 1| NA| 0| NA| NA| NA| NA| NA| YES| YES|
1987| 10| 6| 2| NA| 830| NA| 937| PS| 1457| NA| NA| 67| NA| NA| NA| LAX| SFO| 337| NA| NA| 1| NA| 0| NA| NA| NA| NA| NA| YES| YES|
1987| 10| 29| 4| NA| 930| NA| 1040| PS| 1459| NA| NA| 70| NA| NA| NA| LAX| SFO| 337| NA| NA| 1| NA| 0| NA| NA| NA| NA| NA| YES| YES|
1987| 10| 22| 4| NA| 2000| NA| 2059| PS| 1465| NA| NA| 59| NA| NA| NA| BUR| OAK| 325| NA| NA| 1| NA| 0| NA| NA| NA| NA| NA| YES| YES|
1987| 10| 5| 1| NA| 1845| NA| 2005| PS| 1480| NA| NA| 80| NA| NA| NA| SFO| SAN| 447| NA| NA| 1| NA| 0| NA| NA| NA| NA| NA| YES| YES|
1987| 10| 28| 3| NA| 1930| NA| 2040| PS| 1502| NA| NA| 70| NA| NA| NA| SMF| LAX| 373| NA| NA| 1| NA| 0| NA| NA| NA| NA| NA| YES| YES|
1987| 10| 11| 7| NA| 700| NA| 807| PS| 1517| NA| NA| 67| NA| NA| NA| LAX| SFO| 337| NA| NA| 1| NA| 0| NA| NA| NA| NA| NA| YES| YES|
1987| 10| 1| 4| NA| 1600| NA| 1708| PS| 1522| NA| NA| 68| NA| NA| NA| SFO| LAX| 337| NA| NA| 1| NA| 0| NA| NA| NA| NA| NA| YES| YES|
1987| 10| 23| 5| NA| 1600| NA| 1708| PS| 1522| NA| NA| 68| NA| NA| NA| SFO| LAX| 337| NA| NA| 1| NA| 0| NA| NA| NA| NA| NA| YES| YES|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
only showing top 10 rows

In [0]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

temp_air_df = air_data.select(['Year','Month','Origin','Dest','Distance', 'ArrDelay'])
temp_air_df = temp_df.withColumn('ArrDelay', temp_air_df.ArrDelay.cast(IntegerType()))# randomly selected rows
window_spec = Window().partitionBy(['Dest', 'Origin', 'Month']).orderBy(F.desc('ArrDelay'))
temp_df.withColumn('rank', F.rank().over(window_spec)).show(100)

+----+-----+------+----+--------+--------+----+
Year|Month|Origin|Dest|Distance|ArrDelay|rank|
+----+-----+------+----+--------+--------+----+
1987| 10| DTW| ABE| 424| 55| 1|
1987| 10| DTW| ABE| 424| 28| 2|
1987| 10| DTW| ABE| 424| 25| 3|
1987| 10| DTW| ABE| 424| 24| 4|
1987| 10| DTW| ABE| 424| 23| 5|
1987| 10| DTW| ABE| 424| 22| 6|
1987| 10| DTW| ABE| 424| 20| 7|
1987| 10| DTW| ABE| 424| 20| 7|
1987| 10| DTW| ABE| 424| 19| 9|
1987| 10| DTW| ABE| 424| 18| 10|
1987| 10| DTW| ABE| 424| 16| 11|
1987| 10| DTW| ABE| 424| 16| 11|
1987| 10| DTW| ABE| 424| 15| 13|
1987| 10| DTW| ABE| 424| 14| 14|
1987| 10| DTW| ABE| 424| 14| 14|
1987| 10| DTW| ABE| 424| 13| 16|
1987| 10| DTW| ABE| 424| 13| 16|
1987| 10| DTW| ABE| 424| 13| 16|
1987| 10| DTW| ABE| 424| 13| 16|
1987| 10| DTW| ABE| 424| 12| 20|
1987| 10| DTW| ABE| 424| 12| 20|
1987| 10| DTW| ABE| 424| 12| 20|
1987| 10| DTW| ABE| 424| 11| 23|
1987| 10| DTW| ABE| 424| 10| 24|
1987| 10| DTW| ABE| 424| 10| 24|
1987| 10| DTW| ABE| 424| 10| 24|
1987| 10| DTW| ABE| 424| 10| 24|
1987| 10| DTW| ABE| 424| 9| 28|
1987| 10| DTW| ABE| 424| 8| 29|
1987| 10| DTW| ABE| 424| 8| 29|
1987| 10| DTW| ABE| 424| 8| 29|
1987| 10| DTW| ABE| 424| 8| 29|
1987| 10| DTW| ABE| 424| 7| 33|
1987| 10| DTW| ABE| 424| 7| 33|
1987| 10| DTW| ABE| 424| 6| 35|
1987| 10| DTW| ABE| 424| 5| 36|
1987| 10| DTW| ABE| 424| 5| 36|
1987| 10| DTW| ABE| 424| 5| 36|
1987| 10| DTW| ABE| 424| 3| 39|
1987| 10| DTW| ABE| 424| 3| 39|
1987| 10| DTW| ABE| 424| 3| 39|
1987| 10| DTW| ABE| 424| 3| 39|
1987| 10| DTW| ABE| 424| 2| 43|
1987| 10| DTW| ABE| 424| 2| 43|
1987| 10| DTW| ABE| 424| 1| 45|
1987| 10| DTW| ABE| 424| 0| 46|
1987| 10| DTW| ABE| 424| 0| 46|
1987| 10| DTW| ABE| 424| 0| 46|
1987| 10| DTW| ABE| 424| 0| 46|
1987| 10| DTW| ABE| 424| 0| 46|
1987| 10| DTW| ABE| 424| 0| 46|
1987| 10| DTW| ABE| 424| 0| 46|
1987| 10| DTW| ABE| 424| 0| 46|
1987| 10| DTW| ABE| 424| 0| 46|
1987| 10| DTW| ABE| 424| 0| 46|
1987| 10| DTW| ABE| 424| 0| 46|
1987| 10| DTW| ABE| 424| -1| 57|
1987| 10| DTW| ABE| 424| -1| 57|
1987| 10| DTW| ABE| 424| -2| 59|
1987| 10| DTW| ABE| 424| -3| 60|
1987| 10| DTW| ABE| 424| -3| 60|
1987| 10| DTW| ABE| 424| -4| 62|
1987| 10| DTW| ABE| 424| -6| 63|
1987| 10| DTW| ABE| 424| -6| 63|
1987| 10| DTW| ABE| 424| -6| 63|
1987| 10| DTW| ABE| 424| -7| 66|
1987| 10| DTW| ABE| 424| -7| 66|
1987| 10| DTW| ABE| 424| -7| 66|
1987| 10| DTW| ABE| 424| -7| 66|
1987| 10| DTW| ABE| 424| -10| 70|
1987| 10| DTW| ABE| 424| -10| 70|
1987| 10| DTW| ABE| 424| -12| 72|
1987| 10| DTW| ABE| 424| -12| 72|
1987| 10| DTW| ABE| 424| -12| 72|
1987| 10| DTW| ABE| 424| -14| 75|
1987| 10| DTW| ABE| 424| -15| 76|
1987| 10| DTW| ABE| 424| -15| 76|
1987| 10| DTW| ABE| 424| -17| 78|
1987| 10| DTW| ABE| 424| null| 79|
1987| 10| DFW| ABQ| 569| 119| 1|
1987| 10| DFW| ABQ| 569| 48| 2|
1987| 10| DFW| ABQ| 569| 45| 3|
1987| 10| DFW| ABQ| 569| 37| 4|
1987| 10| DFW| ABQ| 569| 33| 5|
1987| 10| DFW| ABQ| 569| 32| 6|
1987| 10| DFW| ABQ| 569| 31| 7|
1987| 10| DFW| ABQ| 569| 30| 8|
1987| 10| DFW| ABQ| 569| 30| 8|
1987| 10| DFW| ABQ| 569| 29| 10|
1987| 10| DFW| ABQ| 569| 28| 11|
1987| 10| DFW| ABQ| 569| 28| 11|
1987| 10| DFW| ABQ| 569| 27| 13|
1987| 10| DFW| ABQ| 569| 27| 13|
1987| 10| DFW| ABQ| 569| 27| 13|
1987| 10| DFW| ABQ| 569| 26| 16|
1987| 10| DFW| ABQ| 569| 26| 16|
1987| 10| DFW| ABQ| 569| 26| 16|
1987| 10| DFW| ABQ| 569| 26| 16|
1987| 10| DFW| ABQ| 569| 24| 20|
1987| 10| DFW| ABQ| 569| 24| 20|
+----+-----+------+----+--------+--------+----+
only showing top 100 rows

In [0]:
window_spec2 = Window().orderBy('Date') # can also use this to partition
sdf_apple.withColumn('Volume_7lag', F.lag('Volume',7).over(window_spec2)).show() # a window() function is necessary for lag????

+----------+------------------+------------------+------------------+------------------+---------+------------------+-----------+
 Date| Open| High| Low| Close| Volume| Adj Close|Volume_7lag|
+----------+------------------+------------------+------------------+------------------+---------+------------------+-----------+
2010-01-04| 213.429998| 214.499996|212.38000099999996| 214.009998|123432400| 27.727039| null|
2010-01-05| 214.599998| 215.589994| 213.249994| 214.379993|150476200|27.774976000000002| null|
2010-01-06| 214.379993| 215.23| 210.750004| 210.969995|138040000|27.333178000000004| null|
2010-01-07| 211.75| 212.000006| 209.050005| 210.58|119282800| 27.28265| null|
2010-01-08| 210.299994| 212.000006|209.06000500000002|211.98000499999998|111902700| 27.464034| null|
2010-01-11|212.79999700000002| 213.000002| 208.450005|210.11000299999998|115557400| 27.221758| null|
2010-01-12|209.18999499999998|209.76999500000002| 206.419998| 207.720001|148614900| 26.91211| null|
2010-01-13| 207.870005|210.92999500000002| 204.099998| 210.650002|151473000| 27.29172| 123432400|
2010-01-14|210.11000299999998|210.45999700000002| 209.020004| 209.43|108223500| 27.133657| 150476200|
2010-01-15|210.92999500000002|211.59999700000003| 205.869999| 205.93|148516900|26.680197999999997| 138040000|
2010-01-19| 208.330002|215.18999900000003| 207.240004| 215.039995|182501900|27.860484999999997| 119282800|
2010-01-20| 214.910006| 215.549994| 209.500002| 211.73|153038200| 27.431644| 111902700|
2010-01-21| 212.079994|213.30999599999998| 207.210003| 208.069996|152038600| 26.957455| 115557400|
2010-01-22|206.78000600000001| 207.499996| 197.16| 197.75|220441900| 25.620401| 148614900|
2010-01-25|202.51000200000001| 204.699999| 200.190002| 203.070002|266424900|26.309658000000002| 151473000|
2010-01-26|205.95000100000001| 213.710005| 202.580004| 205.940001|466777500| 26.681494| 108223500|
2010-01-27| 206.849995| 210.58| 199.530001| 207.880005|430642100|26.932840000000002| 148516900|
2010-01-28| 204.930004| 205.500004| 198.699995| 199.289995|293375600|25.819922000000002| 182501900|
2010-01-29| 201.079996| 202.199995| 190.250002| 192.060003|311488100| 24.883208| 153038200|
2010-02-01|192.36999699999998| 196.0|191.29999899999999| 194.729998|187469100| 25.229131| 152038600|
+----------+------------------+------------------+------------------+------------------+---------+------------------+-----------+
only showing top 20 rows

In [0]:
window_spec3 = Window().orderBy('Date').rowsBetween(-6,0) # we can use (Window.unboundedPreceding, Window.currentRow) instead of (-6,0) to use all previous rows
sdf_apple = sdf_apple.withColumnRenamed('Adj Close', 'Adj_close')
sdf_apple = sdf_apple.withColumn('Adj_close',sdf_apple.Adj_close.cast(IntegerType()))# converting Adj Close column to int for ease
sdf_apple = sdf_apple.withColumn('Adj_close_rolling', F.sum('Adj_close').over(window_spec3))
sdf_apple.show()

+----------+------------------+------------------+------------------+------------------+---------+---------+-----------------+
 Date| Open| High| Low| Close| Volume|Adj_close|Adj_close_rolling|
+----------+------------------+------------------+------------------+------------------+---------+---------+-----------------+
2010-01-04| 213.429998| 214.499996|212.38000099999996| 214.009998|123432400| 27| 27|
2010-01-05| 214.599998| 215.589994| 213.249994| 214.379993|150476200| 27| 54|
2010-01-06| 214.379993| 215.23| 210.750004| 210.969995|138040000| 27| 81|
2010-01-07| 211.75| 212.000006| 209.050005| 210.58|119282800| 27| 108|
2010-01-08| 210.299994| 212.000006|209.06000500000002|211.98000499999998|111902700| 27| 135|
2010-01-11|212.79999700000002| 213.000002| 208.450005|210.11000299999998|115557400| 27| 162|
2010-01-12|209.18999499999998|209.76999500000002| 206.419998| 207.720001|148614900| 26| 188|
2010-01-13| 207.870005|210.92999500000002| 204.099998| 210.650002|151473000| 27| 188|
2010-01-14|210.11000299999998|210.45999700000002| 209.020004| 209.43|108223500| 27| 188|
2010-01-15|210.92999500000002|211.59999700000003| 205.869999| 205.93|148516900| 26| 187|
2010-01-19| 208.330002|215.18999900000003| 207.240004| 215.039995|182501900| 27| 187|
2010-01-20| 214.910006| 215.549994| 209.500002| 211.73|153038200| 27| 187|
2010-01-21| 212.079994|213.30999599999998| 207.210003| 208.069996|152038600| 26| 186|
2010-01-22|206.78000600000001| 207.499996| 197.16| 197.75|220441900| 25| 185|
2010-01-25|202.51000200000001| 204.699999| 200.190002| 203.070002|266424900| 26| 184|
2010-01-26|205.95000100000001| 213.710005| 202.580004| 205.940001|466777500| 26| 183|
2010-01-27| 206.849995| 210.58| 199.530001| 207.880005|430642100| 26| 183|
2010-01-28| 204.930004| 205.500004| 198.699995| 199.289995|293375600| 25| 181|
2010-01-29| 201.079996| 202.199995| 190.250002| 192.060003|311488100| 24| 178|
2010-02-01|192.36999699999998| 196.0|191.29999899999999| 194.729998|187469100| 25| 177|
+----------+------------------+------------------+------------------+------------------+---------+---------+-----------------+
only showing top 20 rows

In [0]:
# count the number of cancelled flights to each destination on a specific date

temp_air_df = air_data.select('Year', 'Month', 'DayOfMonth', 'FlightNum', 'Cancelled', 'Origin', 'Dest')
#emp_air_df.show()

In [0]:

air_df_pivot = temp_air_df.groupBy('Year','Month','DayOfMonth').pivot('Dest')\
            .agg(F.sum('Cancelled').alias('Cancelled'), (F.count('Cancelled')-F.sum('Cancelled')).alias('NotCancelled')) # one way to do it, but it works only when Cancelled is either 0 or 1

# what is a better way to do this?
air_df_pivot.show(1)

+----+-----+----------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+-------------+----------------+---------

In [0]:
# there are two count functions available:
# pyspark.sql.Functions.count is count(col), it operates on a column in dataframe
# df.count() operates on entire dataframe and returns the number of rows in the data frame

In [0]:

# also sdf['col_name'] gives a column while sdf.select('col_name') gives a dataframe

temp_air_df.tail(1) # selecting top or bottom rows, using head or tail

Out[131]: [Row(Year=1987, Month=11, DayOfMonth=28, FlightNum=542, Cancelled=0, Origin='CLT', Dest='BOS')]

In [0]:
# just verifying the results
temp_air_df.filter( (temp_air_df.Dest == 'ABE') & (temp_air_df.Year == 1987) & (temp_air_df.Month == 10) & (temp_air_df.DayOfMonth ==3) &  (temp_air_df.Cancelled ==0)).count()
# how to select first row from the dataframe?


Out[126]: 14

In [0]:
help(F.getRows)

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-1841341981606068> in <module> 
 ----> 1 help ( F . getRows ) 

 AttributeError : module 'pyspark.sql.functions' has no attribute 'getRows'